In [ ]:
# Использование предобученной модели из transformers для получения embeddings
from transformers import AutoModel, AutoTokenizer
import torch

# Загружаем предобученную модель и токенизатор
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Подготавливаем текст
text = "Пример текста для получения векторного представления"
inputs = tokenizer(text, return_tensors="pt")

# Получаем embeddings (последний скрытый слой)
with torch.no_grad():
    outputs = model(**inputs)
    
# Берем состояние последнего слоя
last_hidden_state = outputs.last_hidden_state

# Для представления всего предложения можно использовать среднее значение
sentence_embedding = torch.mean(last_hidden_state, dim=1)

print(f"Размерность embedding: {sentence_embedding.shape}")
print(f"Embedding: {sentence_embedding[0][:5]}")  # Показываем первые 5 значений

In [ ]:
import torch
import numpy as np
from transformers import AutoModel, AutoTokenizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Функция для получения embeddings из BERT
def get_bert_embeddings(texts, model, tokenizer, device="cpu"):
    embeddings = []
    
    for text in texts:
        # Токенизация
        encoded_input = tokenizer(text, padding=True, truncation=True, 
                                  max_length=128, return_tensors='pt')
        # Перемещаем на нужное устройство
        encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
        
        # Получаем выходные данные модели
        with torch.no_grad():
            model_output = model(**encoded_input)
            
        # Извлекаем последний скрытый слой
        last_hidden_state = model_output.last_hidden_state
        
        # Используем [CLS] токен (первый токен) как представление предложения
        sentence_embedding = last_hidden_state[:, 0, :].cpu().numpy()
        embeddings.append(sentence_embedding[0])
    
    return np.array(embeddings)

# Загрузка модели
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Пример данных
texts = [
    "I love this movie, it's amazing!",
    "This film is terrible, I hated it.",
    "What a great movie, I enjoyed every minute.",
    "Absolutely disappointing, waste of time.",
    "This is the best film I've seen in years.",
    "The worst movie ever, avoid at all costs."
]
labels = [1, 0, 1, 0, 1, 0]  # 1 - положительный, 0 - отрицательный

# Получаем embeddings
embeddings = get_bert_embeddings(texts, model, tokenizer, device)

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    embeddings, labels, test_size=0.3, random_state=42
)

# Обучаем простой классификатор
clf = LogisticRegression()
clf.fit(X_train, y_train)

# Оцениваем результаты
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:")
print(report)

# Классифицируем новый текст
new_text = "I watched this movie yesterday and really liked it."
new_embedding = get_bert_embeddings([new_text], model, tokenizer, device)
prediction = clf.predict(new_embedding)
print(f"Predicted sentiment for new text: {'Positive' if prediction[0] == 1 else 'Negative'}")

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from transformers import AutoModel, AutoTokenizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

class AdvancedEmbeddingExtractor:
    def __init__(self, model_name="bert-base-uncased", device=None):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name, output_hidden_states=True)
        self.device = device if device else torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)
        self.model.eval()
        
        # Веса для разных слоев (можно оптимизировать под конкретную задачу)
        # Здесь мы даем больший вес верхним слоям
        num_layers = self.model.config.num_hidden_layers + 1  # +1 для embedding слоя
        self.layer_weights = torch.nn.Parameter(
            torch.tensor([i/num_layers for i in range(num_layers)], device=self.device)
        )
        self.layer_weights = torch.softmax(self.layer_weights, dim=0)
        
    def get_embeddings(self, texts, pooling_strategy="mean", layers="all"):
        """
        Получение embeddings для текстов с разными стратегиями объединения.
        
        Args:
            texts: Список текстов
            pooling_strategy: Стратегия объединения токенов ('cls', 'mean', 'max', 'attention')
            layers: Какие слои использовать ('all', 'last', or list of indices)
            
        Returns:
            Numpy array с embeddings
        """
        batch_embeddings = []
        
        for text in texts:
            # Токенизация
            inputs = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
            inputs = {k: v.to(self.device) for k, v in inputs.items()}
            
            # Получаем выходы со всех слоев
            with torch.no_grad():
                outputs = self.model(**inputs)
                
            # Получаем hidden states со всех слоев
            hidden_states = outputs.hidden_states
            
            # Выбираем слои для использования
            if layers == "last":
                selected_layers = [hidden_states[-1]]
            elif layers == "all":
                selected_layers = hidden_states
            else:
                selected_layers = [hidden_states[i] for i in layers]
                
            # Объединяем выбранные слои с весами
            if len(selected_layers) == 1:
                combined_states = selected_layers[0]
            else:
                # Используем веса для слоев
                weights = self.layer_weights[-len(selected_layers):]
                weights = weights / weights.sum()  # Нормализуем веса
                
                combined_states = torch.zeros_like(selected_layers[0])
                for i, layer in enumerate(selected_layers):
                    combined_states += layer * weights[i].item()
            
            # Применяем стратегию объединения токенов
            if pooling_strategy == "cls":
                # Используем токен [CLS] (первый токен)
                embedding = combined_states[0, 0, :]
            elif pooling_strategy == "mean":
                # Среднее по всем токенам (исключая padding)
                attention_mask = inputs['attention_mask']
                embedding = self._mean_pooling(combined_states, attention_mask)
            elif pooling_strategy == "max":
                # Максимальное значение по каждой размерности
                embedding = torch.max(combined_states[0], dim=0)[0]
            elif pooling_strategy == "attention":
                # Weighted pooling с использованием self-attention
                embedding = self._attention_pooling(combined_states[0], outputs.attentions[-1])
            else:
                raise ValueError(f"Unsupported pooling strategy: {pooling_strategy}")
                
            batch_embeddings.append(embedding.cpu().numpy())
            
        return np.array(batch_embeddings)
    
    def _mean_pooling(self, token_embeddings, attention_mask):
        # Среднее значение по токенам, с учетом маски внимания
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
        sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        return sum_embeddings / sum_mask
    
    def _attention_pooling(self, token_embeddings, attention):
        # Используем веса из последнего слоя внимания для объединения токенов
        # Берем среднее по головам внимания
        mean_attention = torch.mean(attention[0], dim=0)
        # Используем веса из первой строки (соответствующей [CLS] токену)
        cls_attention_weights = mean_attention[0]
        weighted_sum = torch.matmul(cls_attention_weights.unsqueeze(0), token_embeddings)
        return weighted_sum.squeeze(0)
    
    def visualize_embeddings(self, texts, labels=None):
        """
        Визуализация embeddings с помощью t-SNE
        """
        from sklearn.manifold import TSNE
        
        embeddings = self.get_embeddings(texts)
        
        # Применяем t-SNE для уменьшения размерности до 2D
        tsne = TSNE(n_components=2, random_state=42)
        embeddings_2d = tsne.fit_transform(embeddings)
        
        # Визуализация
        plt.figure(figsize=(10, 8))
        scatter = plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], 
                             c=labels if labels else None)
        
        if labels is not None and len(set(labels)) < 10:
            # Добавляем легенду, если количество уникальных меток небольшое
            plt.legend(handles=scatter.legend_elements()[0], 
                      labels=[f"Class {i}" for i in set(labels)])
        
        # Добавляем аннотации с текстами
        for i, txt in enumerate(texts):
            # Сокращаем текст для отображения
            short_txt = txt[:20] + "..." if len(txt) > 20 else txt
            plt.annotate(short_txt, (embeddings_2d[i, 0], embeddings_2d[i, 1]))
            
        plt.title("t-SNE визуализация текстовых embeddings")
        plt.tight_layout()
        plt.show()
        
    def semantic_search(self, query, corpus, top_k=5):
        """
        Семантический поиск на основе embeddings
        """
        query_embedding = self.get_embeddings([query])[0]
        corpus_embeddings = self.get_embeddings(corpus)
        
        # Вычисляем косинусное сходство
        similarities = cosine_similarity([query_embedding], corpus_embeddings)[0]
        
        # Сортируем результаты
        results = [(i, similarities[i], corpus[i]) for i in range(len(similarities))]
        results.sort(key=lambda x: x[1], reverse=True)
        
        return results[:top_k]

# Пример использования
extractor = AdvancedEmbeddingExtractor()

texts = [
    "Machine learning is a field of study in artificial intelligence.",
    "Deep learning is a subset of machine learning.",
    "Neural networks are the foundation of deep learning.",
    "Computer vision is an application of deep learning.",
    "Natural language processing deals with text data.",
    "Python is the most popular programming language for AI.",
    "The Transformer architecture revolutionized NLP tasks.",
    "GPT and BERT are examples of large language models.",
    "Reinforcement learning is learning by trial and error.",
    "Unsupervised learning doesn't require labeled data."
]

labels = [0, 0, 0, 1, 1, 2, 3, 3, 4, 4]  # Категории текстов

# Получаем embeddings с разными стратегиями
cls_embeddings = extractor.get_embeddings(texts, pooling_strategy="cls")
mean_embeddings = extractor.get_embeddings(texts, pooling_strategy="mean")
attention_embeddings = extractor.get_embeddings(texts, pooling_strategy="attention", layers="all")

# Пример семантического поиска
query = "How do language models work?"
search_results = extractor.semantic_search(query, texts, top_k=3)

print("Семантический поиск для запроса:", query)
for idx, score, text in search_results:
    print(f"Score: {score:.4f} - {text}")

# Визуализация embeddings
extractor.visualize_embeddings(texts, labels)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

# Подготовка данных
text = """
Artificial intelligence is intelligence demonstrated by machines, as opposed to intelligence of humans.
The term "artificial intelligence" is often used to describe machines that mimic cognitive functions 
that humans associate with the human mind, such as learning and problem solving. As machines become 
increasingly capable, tasks considered to require intelligence are often removed from the definition of AI.
"""

# Токенизация и создание словаря
words = word_tokenize(text.lower())
vocab = Counter(words)
vocab = {word: i for i, (word, _) in enumerate(vocab.most_common())}
vocab_size = len(vocab)

# Параметры модели
embedding_dim = 10
context_size = 2

# Создаем пары контекст-целевое слово
data = []
for i in range(context_size, len(words) - context_size):
    context = words[i-context_size:i] + words[i+1:i+context_size+1]
    target = words[i]
    data.append((context, target))

# Простая модель CBOW (Continuous Bag of Words)
class CBOW(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear = nn.Linear(embedding_dim, vocab_size)
        
    def forward(self, inputs):
        embeds = sum(self.embeddings(inputs))
        out = self.linear(embeds)
        log_probs = torch.log_softmax(out, dim=0)
        return log_probs

model = CBOW(vocab_size, embedding_dim)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# Обучение модели
for epoch in range(100):
    total_loss = 0
    for context_words, target_word in data:
        # Преобразуем слова в индексы
        context_idxs = torch.tensor([vocab[w] for w in context_words], dtype=torch.long)
        target_idx = torch.tensor([vocab[target_word]], dtype=torch.long)
        
        # Обнуляем градиенты
        model.zero_grad()
        
        # Прямой проход
        log_probs = model(context_idxs)
        
        # Вычисляем потерю
        loss = loss_function(log_probs.unsqueeze(0), target_idx)
        
        # Обратное распространение
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {total_loss}")

# Извлекаем embeddings
word_embeddings = {}
for word, idx in vocab.items():
    word_embeddings[word] = model.embeddings.weight[idx].detach().numpy()

# Пример использования
def get_most_similar(word, top_n=3):
    if word not in vocab:
        return "Слово не найдено в словаре"
    
    word_vec = word_embeddings[word]
    similarities = {}
    
    for w, vec in word_embeddings.items():
        if w != word:
            # Косинусное сходство
            similarity = np.dot(word_vec, vec) / (np.linalg.norm(word_vec) * np.linalg.norm(vec))
            similarities[w] = similarity
    
    # Сортируем по сходству
    sorted_words = sorted(similarities.items(), key=lambda x: x[1], reverse=True)
    return sorted_words[:top_n]

# Проверяем результаты
print("\nНаиболее похожие слова:")
similar_words = get_most_similar("intelligence")
for word, similarity in similar_words:
    print(f"{word}: {similarity:.4f}")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

# Создаем простой датасет для обучения
sentences = [
    "I love machine learning",
    "Deep learning is a subset of machine learning",
    "Neural networks are powerful",
    "Transformer models improved NLP",
    "BERT is a transformer model",
    "GPT is also a transformer model",
    "Python is great for machine learning",
    "I enjoy programming in Python",
    "Data science uses machine learning",
    "Artificial intelligence includes machine learning",
    "Computers can learn from data",
    "Algorithms power machine learning"
]

labels = [0, 0, 0, 1, 1, 1, 2, 2, 0, 0, 0, 0]  # 0: ML, 1: Transformers, 2: Python

# Подготовка данных
all_words = []
for sentence in sentences:
    all_words.extend(word_tokenize(sentence.lower()))

# Строим словарь
word_counts = Counter(all_words)
word_to_idx = {word: i+1 for i, (word, _) in enumerate(word_counts.most_common())}
word_to_idx['<PAD>'] = 0  # Добавляем padding token
vocab_size = len(word_to_idx)

# Максимальная длина предложения
max_length = max(len(word_tokenize(sentence)) for sentence in sentences)

# Функция для преобразования предложения в последовательность индексов
def tokenize_sentence(sentence, max_len):
    tokens = word_tokenize(sentence.lower())
    indices = [word_to_idx.get(token, 0) for token in tokens]
    
    # Дополняем или обрезаем до max_length
    if len(indices) < max_len:
        indices = indices + [0] * (max_len - len(indices))
    else:
        indices = indices[:max_len]
    
    return torch.tensor(indices, dtype=torch.long)

# Подготавливаем данные для обучения
X = [tokenize_sentence(sentence, max_length) for sentence in sentences]
X = torch.stack(X)
y = torch.tensor(labels, dtype=torch.long)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Создаем более продвинутую модель с BiLSTM и Self-Attention
class SentenceEmbeddingModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_classes):
        super(SentenceEmbeddingModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.attention = nn.Linear(hidden_dim * 2, 1)
        self.fc = nn.Linear(hidden_dim * 2, num_classes)
        self.dropout = nn.Dropout(0.3)
        
    def get_embedding(self, x):
        # x: (batch_size, seq_length)
        embedded = self.embedding(x)  # (batch_size, seq_length, embedding_dim)
        
        # Пропускаем через BiLSTM
        lstm_out, _ = self.lstm(embedded)  # (batch_size, seq_length, hidden_dim*2)
        
        # Self-attention механизм
        attention_weights = F.softmax(self.attention(lstm_out), dim=1)
        context_vector = torch.sum(attention_weights * lstm_out, dim=1)  # (batch_size, hidden_dim*2)
        
        return context_vector
        
    def forward(self, x):
        # Получаем embedding предложения
        sentence_embedding = self.get_embedding(x)
        
        # Пропускаем через dropout и классификатор
        sentence_embedding = self.dropout(sentence_embedding)
        logits = self.fc(sentence_embedding)
        return logits

# Параметры модели
embedding_dim = 64
hidden_dim = 32
num_classes = 3
learning_rate = 0.001
num_epochs = 100

# Создаем и обучаем модель
model = SentenceEmbeddingModel(vocab_size, embedding_dim, hidden_dim, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Обучение
losses = []
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    # Прямой проход
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    
    # Обратное распространение
    loss.backward()
    optimizer.step()
    
    losses.append(loss.item())
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Оценка модели
model.eval()
with torch.no_grad():
    outputs = model(X_test)
    _, predicted = torch.max(outputs.data, 1)
    accuracy = (predicted == y_test).sum().item() / y_test.size(0)
    print(f'Точность на тестовой выборке: {accuracy:.4f}')

# Функция для получения embedding предложения
def get_sentence_embedding(sentence):
    model.eval()
    with torch.no_grad():
        tokens = tokenize_sentence(sentence, max_length).unsqueeze(0)
        embedding = model.get_embedding(tokens)
        return embedding.numpy()[0]

# Визуализация embeddings с помощью t-SNE
from sklearn.manifold import TSNE

# Получаем embeddings для всех предложений
all_embeddings = []
for sentence in sentences:
    embedding = get_sentence_embedding(sentence)
    all_embeddings.append(embedding)

all_embeddings = np.array(all_embeddings)

# Применяем t-SNE
tsne = TSNE(n_components=2, random_state=42)
reduced_embeddings = tsne.fit_transform(all_embeddings)

# Визуализируем
plt.figure(figsize=(10, 8))
for i, (x, y) in enumerate(reduced_embeddings):
    plt.scatter(x, y, color=['blue', 'red', 'green'][labels[i]])
    plt.text(x, y, f"{i}: {sentences[i][:20]}...", fontsize=9)

plt.legend(['ML', 'Transformers', 'Python'])
plt.title('t-SNE визуализация Sentence Embeddings')
plt.tight_layout()
plt.show()

# Проверяем модель на новых предложениях
new_sentences = [
    "I like artificial intelligence",
    "BERT and GPT are popular models",
    "I code in Python every day"
]

print("\nКлассификация новых предложений:")
for sentence in new_sentences:
    # Получаем embedding
    tokens = tokenize_sentence(sentence, max_length).unsqueeze(0)
    
    # Получаем предсказание модели
    model.eval()
    with torch.no_grad():
        output = model(tokens)
        _, predicted = torch.max(output.data, 1)
        
    # Определяем метку класса
    class_name = ['ML', 'Transformers', 'Python'][predicted.item()]
    
    print(f"Предложение: '{sentence}'")
    print(f"Класс: {class_name}\n")

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import math
import numpy as np
import random
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

# Загрузим небольшой корпус текстов для обучения
corpus = [
    "The transformer architecture revolutionized natural language processing.",
    "Self-attention mechanisms allow models to focus on different parts of the input.",
    "BERT is a bidirectional encoder representation from transformers.",
    "GPT models are autoregressive transformers for text generation.",
    "Embeddings represent words as dense vectors in a continuous space.",
    "Contextualized embeddings capture word meaning based on surrounding context.",
    "Traditional word embeddings like Word2Vec use static representations.",
    "Language models are trained to predict words based on context.",
    "Machine learning models require numerical representations of text.",
    "Deep learning has become the dominant approach in NLP.",
    "Neural networks consist of layers of interconnected neurons.",
    "Attention is all you need is a famous paper introducing transformers.",
    "Transfer learning allows reuse of pretrained representations.",
    "Fine-tuning adapts pretrained models to specific downstream tasks.",
    "Tokenization is the process of splitting text into smaller units.",
    "Subword tokenization helps handle out-of-vocabulary words.",
    "Word embeddings capture semantic relationships between words.",
    "Transformer models process input in parallel rather than sequentially.",
    "Positional encodings help transformers understand word order.",
    "Multi-head attention computes attention multiple times in parallel."
]

# Подготовка данных
all_sentences = [s.lower() for s in corpus]
all_words = []
for sentence in all_sentences:
    all_words.extend(word_tokenize(sentence))

# Создаем словарь
word_counts = Counter(all_words)
word_to_idx = {
    '<PAD>': 0,  # Padding token
    '<UNK>': 1,  # Unknown token
    '<CLS>': 2,  # Classification token (начало предложения)
    '<SEP>': 3,  # Separator token (конец предложения)
    '<MASK>': 4  # Mask token для MLM задачи
}

# Добавляем слова с частотой больше 1
for word, count in word_counts.items():
    if count > 1 and word not in word_to_idx:
        word_to_idx[word] = len(word_to_idx)

idx_to_word = {idx: word for word, idx in word_to_idx.items()}
vocab_size = len(word_to_idx)
print(f"Размер словаря: {vocab_size}")

# Функция для токенизации и преобразования предложения в индексы
def tokenize_and_encode(sentence, max_len=32):
    tokens = ['<CLS>'] + word_tokenize(sentence.lower()) + ['<SEP>']
    # Преобразуем в индексы, неизвестные слова заменяем на <UNK>
    token_ids = [word_to_idx.get(token, word_to_idx['<UNK>']) for token in tokens]
    
    # Проверяем длину
    if len(token_ids) > max_len:
        token_ids = token_ids[:max_len]
    else:
        # Добавляем padding
        token_ids = token_ids + [word_to_idx['<PAD>']] * (max_len - len(token_ids))
    
    # Создаем маску внимания (1 для реальных токенов, 0 для padding)
    attention_mask = [1 if token_id != word_to_idx['<PAD>'] else 0 for token_id in token_ids]
    
    return token_ids, attention_mask

# Класс для позиционного кодирования
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        
        # Создаем матрицу позиционного кодирования
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        
        # Регистрируем буфер (не параметр, не будет обучаться)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        # x: [batch_size, seq_len, embedding_dim]
        return x + self.pe[:, :x.size(1)]

# Класс для механизма Self-Attention
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        
        assert self.head_dim * num_heads == d_model, "d_model must be divisible by num_heads"
        
        self.query = nn.Linear(d_model, d_model)
        self.key = nn.Linear(d_model, d_model)
        self.value = nn.Linear(d_model, d_model)
        
        self.fc_out = nn.Linear(d_model, d_model)
        
    def forward(self, query, key, value, mask=None):
        batch_size = query.shape[0]
        
        # Linear projections
        Q = self.query(query)  # (batch_size, seq_len, d_model)
        K = self.key(key)      # (batch_size, seq_len, d_model)
        V = self.value(value)  # (batch_size, seq_len, d_model)
        
        # Reshape для multi-head attention
        Q = Q.view(batch_size, -1, self.num_heads, self.head_dim).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, self.num_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.num_heads, self.head_dim).permute(0, 2, 1, 3)
        
        # Вычисляем скалярное произведение и масштабируем
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / math.sqrt(self.head_dim)
        
        # Маскируем padding, если mask задана
        if mask is not None:
            # Преобразуем маску для broadcast по батчу и числу голов
            mask = mask.unsqueeze(1).unsqueeze(2)  # (batch_size, 1, 1, seq_len)
            energy = energy.masked_fill(mask == 0, float("-1e20"))
        
        # Применяем softmax и вычисляем взвешенную сумму
        attention = F.softmax(energy, dim=-1)  # (batch_size, num_heads, seq_len, seq_len)
        out = torch.matmul(attention, V)  # (batch_size, num_heads, seq_len, head_dim)
        
        # Конкатенируем головы и пропускаем через финальный линейный слой
        out = out.permute(0, 2, 1, 3).contiguous().view(batch_size, -1, self.d_model)
        out = self.fc_out(out)  # (batch_size, seq_len, d_model)
        
        return out, attention

# Feed-Forward Network
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(FeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        # x: [batch_size, seq_len, d_model]
        x = F.gelu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Encoder Layer
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super(EncoderLayer, self).__init__()
        self.self_attention = MultiHeadAttention(d_model, num_heads)
        self.norm1 = nn.LayerNorm(d_model)
        self.feed_forward = FeedForward(d_model, d_ff, dropout)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask=None):
        # Self Attention
        attention_out, _ = self.self_attention(x, x, x, mask)
        x = self.norm1(x + self.dropout(attention_out))
        
        # Feed Forward
        ff_out = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_out))
        
        return x

# Полная модель Transformer Encoder
class TransformerEncoder(nn.Module):
    def __init__(self, vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_len=128, dropout=0.1):
        super(TransformerEncoder, self).__init__()
        self.d_model = d_model
        
        # Embedding слой
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_len)
        self.dropout = nn.Dropout(dropout)
        
        # Encoder layers
        self.layers = nn.ModuleList([
            EncoderLayer(d_model, num_heads, d_ff, dropout)
            for _ in range(num_layers)
        ])
        
        # Layer Norm
        self.norm = nn.LayerNorm(d_model)
        
    def forward(self, x, mask=None):
        # x: [batch_size, seq_len]
        
        # Получаем embeddings и добавляем позиционное кодирование
        x = self.embedding(x) * math.sqrt(self.d_model)
        x = self.positional_encoding(x)
        x = self.dropout(x)
        
        # Проходим через энкодер слои
        for layer in self.layers:
            x = layer(x, mask)
            
        # Применяем финальную нормализацию
        x = self.norm(x)
        
        return x

# Создаем модель для Masked Language Modeling (MLM)
class MLMModel(nn.Module):
    def __init__(self, vocab_size, d_model=128, num_heads=2, num_layers=2, d_ff=256, dropout=0.1):
        super(MLMModel, self).__init__()
        
        self.transformer = TransformerEncoder(
            vocab_size=vocab_size,
            d_model=d_model,
            num_heads=num_heads,
            num_layers=num_layers,
            d_ff=d_ff,
            dropout=dropout
        )
        
        # Проекция для MLM задачи
        self.mlm_head = nn.Linear(d_model, vocab_size)
        
    def forward(self, x, mask=None):
        # x: [batch_size, seq_len]
        
        # Получаем contextualized embeddings
        transformer_output = self.transformer(x, mask)  # [batch_size, seq_len, d_model]
        
        # Предсказываем вероятности токенов
        mlm_output = self.mlm_head(transformer_output)  # [batch_size, seq_len, vocab_size]
        
        return mlm_output
    
    def get_embeddings(self, x, mask=None, layer_idx=-1):
        """
        Извлекает embeddings из указанного слоя трансформера
        """
        # Получаем embeddings и добавляем позиционное кодирование
        embeddings = self.transformer.embedding(x) * math.sqrt(self.transformer.d_model)
        x = self.transformer.positional_encoding(embeddings)
        x = self.transformer.dropout(x)
        
        # Проходим через слои до указанного
        all_layer_outputs = []
        for i, layer in enumerate(self.transformer.layers):
            x = layer(x, mask)
            all_layer_outputs.append(x)
            
            if i == layer_idx and layer_idx != -1:
                break
        
        # Если нужен последний слой или прошли все слои
        if layer_idx == -1:
            x = self.transformer.norm(x)
            
        return {
            'token_embeddings': x,  # contextualized embeddings
            'input_embeddings': embeddings,  # чистые embeddings без позиционного кодирования
            'all_layer_outputs': all_layer_outputs  # выходы всех промежуточных слоев
        }

# Функция для создания батчей данных для MLM задачи
def create_mlm_batch(sentences, max_len=32, mask_prob=0.15):
    batch_inputs = []
    batch_labels = []
    batch_masks = []
    
    for sentence in sentences:
        token_ids, attention_mask = tokenize_and_encode(sentence, max_len)
        
        # Копируем исходные токены для labels
        labels = token_ids.copy()
        
        # Случайно маскируем токены
        for i in range(len(token_ids)):
            # Не маскируем специальные токены
            if token_ids[i] in [word_to_idx['<PAD>'], word_to_idx['<CLS>'], word_to_idx['<SEP>']]:
                continue
                
            prob = random.random()
            if prob < mask_prob:
                # 80% случаев заменяем на [MASK]
                if prob < mask_prob * 0.8:
                    token_ids[i] = word_to_idx['<MASK>']
                # 10% случаев заменяем на случайный токен
                elif prob < mask_prob * 0.9:
                    token_ids[i] = random.randint(5, vocab_size - 1)  # Исключаем специальные токены
                # 10% случаев оставляем без изменений
        
        batch_inputs.append(token_ids)
        batch_labels.append(labels)
        batch_masks.append(attention_mask)
    
    # Преобразуем в тензоры
    batch_inputs = torch.tensor(batch_inputs)
    batch_labels = torch.tensor(batch_labels)
    batch_masks = torch.tensor(batch_masks)
    
    return batch_inputs, batch_labels, batch_masks

# Функция обучения модели
def train_mlm_model(model, corpus, num_epochs=50, batch_size=4, learning_rate=1e-4):
    # Подготавливаем оптимизатор
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss(ignore_index=word_to_idx['<PAD>'])  # Игнорируем padding при расчете потерь
    
    losses = []
    
    # Разделяем корпус на батчи
    num_batches = (len(corpus) + batch_size - 1) // batch_size
    
    for epoch in range(num_epochs):
        total_loss = 0
        random.shuffle(corpus)  # Перемешиваем данные
        
        for i in range(0, len(corpus), batch_size):
            batch_sentences = corpus[i:i+batch_size]
            inputs, labels, masks = create_mlm_batch(batch_sentences)
            
            # Обнуляем градиенты
            optimizer.zero_grad()
            
            # Прямой проход
            outputs = model(inputs, masks)
            
            # Изменяем форму выходов и меток для расчета потерь
            outputs = outputs.view(-1, vocab_size)
            labels = labels.view(-1)
            
            # Расчет потерь
            loss = criterion(outputs, labels)
            
            # Обратное распространение
            loss.backward()
            
            # Оптимизация
            optimizer.step()
            
            total_loss += loss.item()
        
        avg_loss = total_loss / num_batches
        losses.append(avg_loss)
        
        if (epoch + 1) % 5 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")
    
    # Визуализация потерь
    plt.figure(figsize=(10, 6))
    plt.plot(losses)
    plt.title('MLM Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.grid(True)
    plt.show()
    
    return model

# Создаем и обучаем модель
d_model = 128  # Размерность embeddings
num_heads = 2  # Количество голов в механизме внимания
num_layers = 2  # Количество слоев Encoder
d_ff = 256  # Размерность Feed-Forward слоя

model = MLMModel(
    vocab_size=vocab_size,
    d_model=d_model,
    num_heads=num_heads,
    num_layers=num_layers,
    d_ff=d_ff,
    dropout=0.1
)

# Обучаем модель
model = train_mlm_model(model, corpus, num_epochs=50, batch_size=4, learning_rate=1e-4)

# Получение embeddings из обученной модели
def get_contextual_embeddings(model, sentence, mode='mean', layer_idx=-1):
    """
    Извлекает embeddings предложения из модели
    
    Args:
        model: обученная модель
        sentence: текст предложения
        mode: метод объединения токенов ('cls', 'mean', 'max')
        layer_idx: индекс слоя (по умолчанию -1, последний слой)
    
    Returns:
        numpy array с embedding предложения
    """
    # Токенизация
    token_ids, attention_mask = tokenize_and_encode(sentence)
    inputs = torch.tensor([token_ids])
    masks = torch.tensor([attention_mask])
    
    # Получаем embeddings
    model.eval()
    with torch.no_grad():
        outputs = model.get_embeddings(inputs, masks, layer_idx=layer_idx)
    
    # Извлекаем токенные embeddings
    token_embeddings = outputs['token_embeddings'][0]  # Берем первый элемент батча
    
    # Применяем выбранный метод объединения
    if mode == 'cls':
        # Используем embedding токена [CLS]
        return token_embeddings[0].numpy()
    
    elif mode == 'mean':
        # Среднее по всем токенам, исключая padding
        mask = torch.tensor(attention_mask).unsqueeze(-1)
        return torch.sum(token_embeddings * mask, dim=0) / torch.sum(mask).numpy()
    
    elif mode == 'max':
        # Max pooling по всем токенам
        # Перед max pooling замаскируем padding нулями
        mask = torch.tensor(attention_mask).unsqueeze(-1)
        masked_embeddings = token_embeddings * mask
        return torch.max(masked_embeddings, dim=0)[0].numpy()
    
    else:
        raise ValueError("Unsupported pooling mode")

# Визуализируем embeddings предложений с помощью t-SNE
def visualize_embeddings(model, sentences, labels=None, mode='mean', layer_idx=-1):
    """
    Визуализирует embeddings предложений с помощью t-SNE
    """
    from sklearn.manifold import TSNE
    
    # Получаем embeddings для всех предложений
    embeddings = []
    for sentence in sentences:
        embedding = get_contextual_embeddings(model, sentence, mode=mode, layer_idx=layer_idx)
        embeddings.append(embedding)
    
    # Применяем t-SNE для снижения размерности
    tsne = TSNE(n_components=2, random_state=42)
    reduced_embeddings = tsne.fit_transform(embeddings)
    
    # Визуализируем
    plt.figure(figsize=(12, 10))
    
    if labels is not None:
        unique_labels = set(labels)
        colors = plt.cm.rainbow(np.linspace(0, 1, len(unique_labels)))
        color_map = dict(zip(unique_labels, colors))
        
        for i, (x, y) in enumerate(reduced_embeddings):
            plt.scatter(x, y, color=color_map[labels[i]])
            plt.text(x, y, f"{i}", fontsize=9)
        
        # Создаем легенду
        for label, color in zip(unique_labels, colors):
            plt.scatter([], [], color=color, label=label)
        plt.legend()
    else:
        for i, (x, y) in enumerate(reduced_embeddings):
            plt.scatter(x, y)
            plt.text(x, y, f"{i}", fontsize=9)
    
    plt.title(f't-SNE visualization of sentence embeddings (mode: {mode}, layer: {layer_idx})')
    plt.tight_layout()
    plt.show()
    
    return reduced_embeddings

# Создадим пример с тестовыми предложениями для визуализации
test_sentences = [
    "Embeddings represent words as vectors.",
    "Word vectors capture semantic relationships.",
    "Neural networks process word embeddings.",
    "BERT produces contextualized embeddings.",
    "GPT generates text based on context.",
    "Transformers use self-attention mechanisms.",
    "Language models predict the next word.",
    "Transfer learning reuses pretrained models.",
    "Machine learning requires data preprocessing.",
    "Deep learning has transformed NLP research."
]

# Присвоим категории предложениям
test_labels = [
    "Embeddings", "Embeddings", "Neural Networks", 
    "Models", "Models", "Architecture",
    "Language Models", "Training", "ML Process", "General"
]

# Визуализируем embeddings
visualize_embeddings(model, test_sentences, test_labels, mode='mean')

# Посмотрим, как отличаются представления из разных слоев
visualize_embeddings(model, test_sentences, test_labels, mode='mean', layer_idx=0)

# Функция для семантического поиска
def semantic_search(model, query, corpus, top_k=3, mode='mean'):
    """
    Выполняет семантический поиск: находит предложения из корпуса,
    наиболее похожие на запрос по косинусному сходству embeddings
    """
    # Получаем embedding запроса
    query_embedding = get_contextual_embeddings(model, query, mode=mode)
    
    # Получаем embeddings всех предложений из корпуса
    corpus_embeddings = []
    for sentence in corpus:
        embedding = get_contextual_embeddings(model, sentence, mode=mode)
        corpus_embeddings.append(embedding)
    
    # Вычисляем косинусное сходство
    similarities = []
    for i, embedding in enumerate(corpus_embeddings):
        # Косинусное сходство
        similarity = np.dot(query_embedding, embedding) / (
            np.linalg.norm(query_embedding) * np.linalg.norm(embedding)
        )
        similarities.append((i, similarity, corpus[i]))
    
    # Сортируем по убыванию сходства
    similarities.sort(key=lambda x: x[1], reverse=True)
    
    return similarities[:top_k]

# Пример семантического поиска
query = "How do language models use embeddings?"
search_results = semantic_search(model, query, corpus, top_k=3)

print("\nРезультаты семантического поиска для запроса:", query)
for idx, score, text in search_results:
    print(f"Score: {score:.4f} - {text}")

# Функция для аналогии слов (word analogy)
def word_analogy(model, word_a, word_b, word_c, top_n=5):
    """
    Реализует операцию аналогии слов: word_a относится к word_b, как word_c относится к ?
    Например: king - man + woman = queen
    """
    # Проверяем, есть ли слова в словаре
    for word in [word_a, word_b, word_c]:
        if word not in word_to_idx:
            print(f"Слово '{word}' не найдено в словаре")
            return []
    
    # Получаем embeddings слов
    embedding_a = model.transformer.embedding(torch.tensor([word_to_idx[word_a]])).detach().squeeze().numpy()
    embedding_b = model.transformer.embedding(torch.tensor([word_to_idx[word_b]])).detach().squeeze().numpy()
    embedding_c = model.transformer.embedding(torch.tensor([word_to_idx[word_c]])).detach().squeeze().numpy()
    
    # Вычисляем вектор аналогии: embedding_b - embedding_a + embedding_c
    target_embedding = embedding_b - embedding_a + embedding_c
    
    # Находим ближайшие слова к целевому вектору
    similarities = []
    for word, idx in word_to_idx.items():
        # Пропускаем специальные токены и исходные слова
        if idx < 5 or word in [word_a, word_b, word_c]:
            continue
        
        word_embedding = model.transformer.embedding(torch.tensor([idx])).detach().squeeze().numpy()
        
        # Косинусное сходство
        similarity = np.dot(target_embedding, word_embedding) / (
            np.linalg.norm(target_embedding) * np.linalg.norm(word_embedding)
        )
        similarities.append((word, similarity))
    
    # Сортируем по убыванию сходства
    similarities.sort(key=lambda x: x[1], reverse=True)
    
    return similarities[:top_n]

# Класс для визуализации внимания
def visualize_attention(model, sentence):
    """
    Визуализирует матрицу внимания для предложения
    """
    # Токенизация
    token_ids, attention_mask = tokenize_and_encode(sentence)
    tokens = [idx_to_word[idx] for idx in token_ids if idx in idx_to_word]
    
    inputs = torch.tensor([token_ids])
    mask = torch.tensor([attention_mask])
    
    # Получаем выходы модели с матрицами внимания
    model.eval()
    with torch.no_grad():
        # Получаем embeddings и добавляем позиционное кодирование
        embeddings = model.transformer.embedding(inputs) * math.sqrt(model.transformer.d_model)
        x = model.transformer.positional_encoding(embeddings)
        x = model.transformer.dropout(x)
        
        # Проходим через слои и сохраняем матрицы внимания
        attention_matrices = []
        for layer in model.transformer.layers:
            # Получаем выход self-attention
            attention_out, attention_weights = layer.self_attention(x, x, x, mask)
            x = layer.norm1(x + layer.dropout(attention_out))
            
            # Feed Forward
            ff_out = layer.feed_forward(x)
            x = layer.norm2(x + layer.dropout(ff_out))
            
            # Сохраняем веса внимания
            attention_matrices.append(attention_weights)
    
    # Визуализируем матрицы внимания для каждого слоя и головы
    num_layers = len(model.transformer.layers)
    num_heads = model.transformer.layers[0].self_attention.num_heads
    
    fig, axes = plt.subplots(num_layers, num_heads, figsize=(15, 4 * num_layers))
    if num_layers == 1:
        axes = np.array([axes])
    
    # Максимальная длина предложения для визуализации
    max_token_len = min(len(tokens), 20)  # Ограничиваем для лучшей визуализации
    tokens = tokens[:max_token_len]
    
    for layer_idx in range(num_layers):
        for head_idx in range(num_heads):
            ax = axes[layer_idx, head_idx]
            
            # Извлекаем матрицу внимания для конкретного слоя и головы
            attn_matrix = attention_matrices[layer_idx][0, head_idx, :max_token_len, :max_token_len].cpu().numpy()
            
            # Создаем тепловую карту
            im = ax.imshow(attn_matrix, cmap='viridis')
            
            # Добавляем метки токенов
            ax.set_xticks(range(max_token_len))
            ax.set_yticks(range(max_token_len))
            ax.set_xticklabels(tokens, rotation=90, fontsize=8)
            ax.set_yticklabels(tokens, fontsize=8)
            
            ax.set_title(f"Layer {layer_idx+1}, Head {head_idx+1